In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import NearestNeighbors
import pickle

import warnings
warnings.filterwarnings("ignore")

In [4]:
books = pd.read_csv("BX-Books.csv", sep=";", error_bad_lines=False, encoding="latin-1")

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


In [7]:
users = pd.read_csv("BX-Users.csv", sep=";", error_bad_lines=False, encoding="latin-1")

In [11]:
ratings = pd.read_csv("BX-Book-Ratings.csv", sep=";", error_bad_lines=False, encoding="latin-1")

# Data Understanding

In [8]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [9]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [12]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [13]:
books.shape

(271360, 8)

In [14]:
users.shape

(278858, 3)

In [15]:
ratings.shape

(1149780, 3)

# Data Cleaning

In [16]:
books = books[["ISBN", "Book-Title", "Book-Author", "Year-Of-Publication", "Publisher"]]
books = books.rename(columns={
    "Book-Title":"title", 
    "Book-Author":"author", 
    "Year-Of-Publication":"year", 
    "Publisher":"publisher"})

In [17]:
users = users.rename(columns={
    "User-ID":"user_id", 
    "Location":"location", 
    "Age":"age"})

In [18]:
ratings = ratings.rename(columns={
    "User-ID":"user_id", 
    "Book-Rating":"rating"})

# Text Preprocessing

In [20]:
# OUTOFF 278858 USERS 105283 GAVE THE BOOK RATING

ratings["user_id"].value_counts().shape

(105283,)

In [21]:
# EXTRACTING THOSE USERS WHO HAVE ATLEAST RATED 200 BOOKS

x = ratings["user_id"].value_counts() > 200
x[x].shape     # 899 ARE THE USERS WHICH HAVE ATLEAST RATED 200 BOOKS AND x[x] EXTRACTS THEM
y = x[x].index
y

Int64Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352,
            110973, 235105,
            ...
            260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727,
            268622, 188951],
           dtype='int64', length=899)

In [22]:
# EXTRACTING 899 USERS FROM user_id  

ratings = ratings[ratings["user_id"].isin(y)]
ratings.shape     # 899 USERS RATED INALL 526356 BOOKS

(526356, 3)

In [23]:
book_rating = ratings.merge(books, on="ISBN")
book_rating.head()

,user_id,ISBN,rating,title,author,year,publisher
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc


In [24]:
book_rating.sample(5)

,user_id,ISBN,rating,title,author,year,publisher
251686,227447,0451202244,0,Above the Law: A Novel,J. F. Freedman,2001,Signet Book
459700,212898,0553278541,0,"Mutiny in Space (Choose Your Own Adventure, No...",Edward Packard,1990,Skylark Books
14635,277639,0380001799,7,Never Leave Me (Avon red-and-gold library),Harold Robbins,1988,Harper Mass Market Paperbacks (Mm)
139902,11676,0373711263,8,No Ordinary Child,Darlene Graham,2003,Harlequin
310710,53628,0590189018,0,Hero for Hire (Star Wars Journal),Donna Tauscher,1998,Scholastic


In [25]:
book_rating.shape

(487671, 7)

In [26]:
number_rating = book_rating.groupby("title")["rating"].count().reset_index()
number_rating.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [27]:
# FINDING NUMBER OF RATING FOR EACH BOOK

number_rating = number_rating.rename(columns={"rating":"number_of_rating"})
final_book_rating = book_rating.merge(number_rating, on="title")
final_book_rating.head()

,user_id,ISBN,rating,title,author,year,publisher,number_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82


In [28]:
final_book_rating.shape

(487671, 8)

In [29]:
# EXTRACTING THE BOOKS WHICH ARE ATLEAST GOT 50 RATINGS SO THAT THE MODEL WON'T DEVIATE FROM 1 or 2 BOOKS RATINGS

final_book_rating = final_book_rating[final_book_rating["number_of_rating"] >=50]
final_book_rating.sample(10)
final_book_rating.shape

(61853, 8)

In [30]:
final_book_rating.duplicated(["user_id", "title"]).sum()

2003

In [31]:
final_book_rating.drop_duplicates(["user_id","title"], inplace=True)
final_book_rating.shape

(59850, 8)

In [32]:
final_book_rating.head()

,user_id,ISBN,rating,title,author,year,publisher,number_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82


In [33]:
#TO GET UNIQUE SUGGESTIONS DURING SEARCH

unique = final_book_rating["title"].unique()
unique = pd.DataFrame(unique)
unique.head()

,0
0,Politically Correct Bedtime Stories: Modern Ta...
1,The Poisonwood Bible: A Novel
2,Bel Canto: A Novel
3,One for the Money (Stephanie Plum Novels (Pape...
4,The Secret Garden


In [34]:
book_pivot = final_book_rating.pivot_table(columns="user_id", index="title", values="rating")
book_pivot.shape     # OUTOFF 899, 742 USERS BOOK RATINGS CAME > 50
book_pivot.head()

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN


In [35]:
#REPLACE NAN VALUES WITH ZERO

book_pivot.fillna(0, inplace=True)
book_pivot.head()

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0


In [36]:
# AS MANY ZERO VALUES ARE PRESENT IN book_pivot USING FOLLOWING LIBRARY book_sparse WILL ONLY CONSIDER NON-ZERO VALUES SO AS TO INCREASE COMPUTATION SPEED

from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot)

In [38]:
book_sparse

<742x888 sparse matrix of type '<class 'numpy.float64'>'
	with 14942 stored elements in Compressed Sparse Row format>

# Model Building

In [37]:
model = NearestNeighbors(algorithm="brute")
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [43]:
distances, suggestions = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1), n_neighbors=6)

In [57]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index==book_name)[0][0]
    distances, suggestions = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors=6)
    
    for i in range(len(suggestions)):
        print(book_pivot.index[suggestions[i]][1:])

In [58]:
recommend_book("The Vampire Lestat (Vampire Chronicles, Book II)")

Index(['The Queen of the Damned (Vampire Chronicles (Paperback))',
       'The Tale of the Body Thief (Vampire Chronicles (Paperback))',
       'Fatal Cure', 'Master of the Game', 'Eyes of a Child'],
      dtype='object', name='title')


In [48]:
unique.sample(10)

,0
171,The Phantom Tollbooth
708,The Plains of Passage (Earth's Children (Paper...
78,One Door Away from Heaven
145,"The Vampire Lestat (Vampire Chronicles, Book II)"
294,She's Come Undone (Oprah's Book Club)
609,Seven Habits Of Highly Effective People
165,The Book of Ruth (Oprah's Book Club (Paperback))
630,Shadowfires
82,Two for the Dough
506,What Looks Like Crazy On An Ordinary Day


In [59]:
pickle.dump(book_pivot, open("book_pivot.pkl", "wb"))

In [60]:
pickle.dump(book_sparse, open("book_sparse.pkl", "wb"))

In [61]:
pickle.dump(unique, open("unique.pkl", "wb"))